Based on the [single-cell best practices online book](https://www.sc-best-practices.org/).

Another helpful resource are these galaxy tutorials: [1](https://training.galaxyproject.org/training-material/topics/single-cell/tutorials/scrna-preprocessing-tenx/tutorial.html) [2](https://training.galaxyproject.org/training-material/topics/single-cell/tutorials/scrna-preprocessing/tutorial.html)

Although Cell Ranger is most widely used for scRNA fastq alignment, I chose to use STARsolo since it is more computationally efficient and I am running on my local machine. STARsolo also allows for multi-gene reads and other features not included in Cell Ranger, as noted in the [2021 preprint](https://doi.org/10.1101/2021.05.05.442755).

STAR requires a Linux-like environment, if you are on a Windows machine like me you must first [install Linux with the Windows Subsystem for Linux (WSL)](https://learn.microsoft.com/en-us/windows/wsl/install), which will enable you to open a Ubuntu (WSL) terminal in vscode. See also the [vscode page for Developing in WSL.](https://code.visualstudio.com/docs/remote/wsl)

### STAR installation

Use the following commands in your Linux terminal:

if you don't already have g++ or make
```
sudo apt-get update
sudo apt-get install g++
sudo apt-get install make
```

I also needed to install zlib to compile STAR
```
sudo apt-get install libz-dev
```

Actual STAR installation
```
wget https://github.com/alexdobin/STAR/archive/2.7.11b.tar.gz
tar -xzf 2.7.11b.tar.gz
cd STAR-2.7.11b/source
make STAR
```

### Download necessary files

#### fastq
There are many datasets provided at [10X genomics](https://www.10xgenomics.com/datasets/). I chose [Human Glioblastoma fastqs](https://www.10xgenomics.com/datasets/human-glioblastoma-multiforme-3-v-3-whole-transcriptome-analysis-3-standard-4-0-0) because it uses a recent chemistry and I started my PhD studying neuroscience.

#### genome files
You need a genome file to align to and a gtf file that contains the annotation information for the genome (genes, exons, introns, etc). These are used to generate genome indexes.

Since I am on my local machine, I am using only chromosome 22 to generate my genome indexes. To generate index files for the whole hg38 genome, you need at least 32GB RAM (per the STARsolo developer). 

[Chromosome 22 file](https://ftp.ensembl.org/pub/release-112/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz)

[gtf file](https://ftp.ensembl.org/pub/release-112/gtf/homo_sapiens/Homo_sapiens.GRCh38.112.gtf.gz)

If you have access to an HPC, the full genome is available [here.](http://ftp.ensembl.org/pub/current_fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz) That link is for the primary assembly, which excludes haplotypes and patches, which are contained in the toplevel sequence file. The STARsolo developer recommends using the primary assembly for generating genome indexes. 

#### whitelist file
The whitelist file used here is for single cell 3' v3 chemistry. More information is available on the [10x Genomics site.](https://kb.10xgenomics.com/hc/en-us/articles/115004506263-What-is-a-barcode-whitelist).

[whitelist file](https://zenodo.org/record/3457880/files/3M-february-2018.txt.gz)

### Generate genome indexes

The first step for using STARsolo is to create genome indexes. The genome directory created by this step is used to align the fastq files. 

In your linux terminal, run generate_genome_indexes.sh (type `./generate_genome_indexes.sh`)
This script is available in the scripts folder, code here for reference:

```
STAR-2.7.11b/source/STAR --runThreadN 8 \
--runMode genomeGenerate \
--genomeDir data/genome_directory \
--genomeFastaFiles data/Homo_sapiens.GRCh38.dna.chromosome.22.fa \
--sjdbGTFfile data/Homo_sapiens.GRCh38.112.gtf \
--sjdbOverhang 100
```

### Fastq alignment

You can use [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) to check the quality of your raw fastq files. After checking your fastq files in fastqc, the reads must be mapped to a genome.

Standard naming conventions for scRNA fastq files:
- L001, L002, ... sequencing lane
- R1 cell barcodes (CB)
- R2 cDNA sequences (transcripts)
- I1 Illumina lane info - not required by STARsolo (req for Cell Ranger)

Barcodes in the R1 file are checked against a "whitelist" file (from Cell Ranger) which lists all cell barcodes. This allows STARsolo to assign each cDNA sequence (in R2 file) to a specific cell.

The following code is for one sample. If you have multiple read files you can create a tab separated file to pass to --readFilesIn.

This script is available in the scripts folder (star_align_single_sample.sh), code here for reference:
```
STAR-2.7.11b/source/STAR --runThreadN 8 \
     --genomeDir data/genome_directory \
     --readFilesIn data/Parent_SC3v3_Human_Glioblastoma_fastqs/Parent_SC3v3_Human_Glioblastoma_S1_L001_R1_001.fastq.gz data/Parent_SC3v3_Human_Glioblastoma_fastqs/Parent_SC3v3_Human_Glioblastoma_S1_L001_R2_001.fastq.gz \
     --soloType CB_UMI_Simple \
     --soloCBwhitelist data/3M-february-2018.txt \
     --soloUMIlen 12 \
     --soloCBlen 16 \
     --outFileNamePrefix STAR_results
```
Note: I am using 10x V3 data, for V2 set --soloUMIlen to 10.

### Quality check
Once you have the STARsolo log, you can check the quality of the alignment using multiqc. Multiqc will automatically search the given path for logs and then write an html document aggregating and summarizing them.

In [ ]:
#install multiqc to the notebook kernel
%pip install --quiet multiqc pandas
#reset to restart kernal after install
%reset -f

In [ ]:
import multiqc

multiqc.parse_logs()

In [ ]:
multiqc.write_report()

If you are like me and used only chromosome 22 to generate your gene indexes, note that the QC report is going to show poor quality. Many reads will not be aligned since we only gave STARsolo one chromosome to align to.

### Import data to scanpy
scanpy expects output from cell ranger, so you need to gzip matrix.mtx, barcodes.tsv, features.tsv for scanpy to read them in.

In [ ]:
#import libraries
import scanpy as sc
import anndata as ad

#read data into scanpy
adata = sc.read_10x_mtx("STAR_resultsSolo.out\\Gene\\filtered")
adata

For scRNA data:
- n_obs = barcodes (cells)
- n_vars = transcripts (genes)

Now that we put our results into an AnnData object, you can use your STARsolo output just like any other aligned data! 

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )